In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Data loading and preprocessing for steatosis, inflammation, and ballooning
def load_data(hallmark):
    X = []
    y = []
    
    for label in ['0', '1', '2', '3']:
        path = f"{hallmark}/{label}"
        images = os.listdir(path)
        for image in images:
            img = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (150, 150))
            X.append(img)
            y.append(label)

    X = np.array(X)
    y = to_categorical(y, num_classes=4)
    return X, y

X_steatosis, y_steatosis = load_data("D:/DATASET1/CNN/steatosis/train")
X_inflammation, y_inflammation = load_data("D:/DATASET1/CNN/inflammation/train")
X_ballooning, y_ballooning = load_data("D:/DATASET1/CNN/ballooning/train")

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
# Split data into train and validation sets
X_steatosis_train, X_steatosis_val, y_steatosis_train, y_steatosis_val = train_test_split(X_steatosis, y_steatosis, test_size=0.2, random_state=42)
X_inflammation_train, X_inflammation_val, y_inflammation_train, y_inflammation_val = train_test_split(X_inflammation, y_inflammation, test_size=0.2, random_state=42)
X_ballooning_train, X_ballooning_val, y_ballooning_train, y_ballooning_val = train_test_split(X_ballooning, y_ballooning, test_size=0.2, random_state=42)


In [ ]:
# Data augmentation and resizing for fibrosis
fibrosis_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
fibrosis_generator = fibrosis_datagen.flow_from_directory("D:/DATASET1/CNN/fibrosis/train", target_size=(150, 150), batch_size=32, class_mode='categorical')

In [ ]:
# Model creation and training for steatosis, inflammation, and ballooning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [ ]:
model_steatosis = Model(inputs=base_model.input, outputs=predictions)
model_inflammation = Model(inputs=base_model.input, outputs=predictions)
model_ballooning = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model_steatosis.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_inflammation.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_ballooning.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_steatosis.fit(X_steatosis_train, y_steatosis_train, validation_data=(X_steatosis_val, y_steatosis_val), epochs=10, batch_size=32)
model_inflammation.fit(X_inflammation_train, y_inflammation_train, validation_data=(X_inflammation_val, y_inflammation_val), epochs=10, batch_size=32)
model_ballooning.fit(X_ballooning_train, y_ballooning_train, validation_data=(X_ballooning_val, y_ballooning_val), epochs=10, batch_size=32)


In [ ]:
# Model creation and training for fibrosis
fibrosis_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = fibrosis_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

model_fibrosis = Model(inputs=fibrosis_model.input, outputs=predictions)

for layer in fibrosis_model.layers:
    layer.trainable = False

model_fibrosis.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_fibrosis.fit(fibrosis_generator, epochs=10, steps_per_epoch=len(fibrosis_generator))


In [ ]:
# Fine-tuning
for layer in model_steatosis.layers[:100]:
    layer.trainable = False
for layer in model_inflammation.layers[:100]:
    layer.trainable = False
for layer in model_ballooning.layers[:100]:
    layer.trainable = False
for layer in model_fibrosis.layers[:100]:
    layer.trainable = False

model_steatosis.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_inflammation.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_ballooning.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_fibrosis.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_steatosis.fit(X_steatosis_train, y_steatosis_train, validation_data=(X_steatosis_val, y_steatosis_val), epochs=10, batch_size=32)
model_inflammation.fit(X_inflammation_train, y_inflammation_train, validation_data=(X_inflammation_val, y_inflammation_val), epochs=10, batch_size=32)
model_ballooning.fit(X_ballooning_train, y_ballooning_train, validation_data=(X_ballooning_val, y_ballooning_val), epochs=10, batch_size=32)
model_fibrosis.fit(fibrosis_generator, epochs=10, steps_per_epoch=len(fibrosis_generator))


In [ ]:
# Feature extraction for ANN
features_steatosis_train = model_steatosis.predict(X_steatosis_train)
features_steatosis_val = model_steatosis.predict(X_steatosis_val)

features_inflammation_train = model_inflammation.predict(X_inflammation_train)
features_inflammation_val = model_inflammation.predict(X_inflammation_val)

features_ballooning_train = model_ballooning.predict(X_ballooning_train)
features_ballooning_val = model_ballooning.predict(X_ballooning_val)

features_fibrosis_train = model_fibrosis.predict(fibrosis_generator)
features_fibrosis_val = model_fibrosis.predict(fibrosis_generator)